# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "data/nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU","BPXDI1"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [10]:
# enter your code here
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

0.031. Almost same.

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [11]:
# Create a labeled version of the gender variable
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

model = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC drops from 0.031 to 0.030

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [12]:
da_male = da[da['RIAGENDRx']=='Male']
da_female = da[da['RIAGENDRx']=='Female']

model_male = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_male)
result_male = model_male.fit()
print('Male:',result_male.cov_struct.summary())

model_female = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_female)
result_female = model_female.fit()
print('Female:',result_female.cov_struct.summary())

Male: The correlation between two observations in the same cluster is 0.035
Female: The correlation between two observations in the same cluster is 0.029


__Q3a.__ What do you learn by comparing these two fitted models?

There is higher correlation in data among males than females

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [21]:
# Relabel the levels, convert rare categories to missing.
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU
da.head()
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2x)", groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
======================================================================
Model:                 MixedLM     Dependent Variable:     BPXDI1     
No. Observations:      5100        Method:                 REML       
No. Groups:            30          Scale:                  154.2352   
Min. group size:       105         Likelihood:             -20111.9561
Max. group size:       226         Converged:              Yes        
Mean group size:       170.0                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                   65.646    1.032 63.631 0.000 63.624 67.668
RIAGENDRx[T.Male]            2.755    0.351  7.855 0.000  2.068  3.443
C(DMDEDUC2x)[T.HS]          -1.093    0.521 -2.098 0.036 -2.115 -0.072
C(DMDEDUC2x)[T.SomeCollege] -0.428    0.484 -0.883 0.377 -1.377  0.521
C(DMDEDUC2x)[T.lt9]         -0.955    0.641 -1.488 0.137 -2.212  0.303
C(DMDEDUC2x)[T.x9_11]       -0.219    0.630 -0.347 0.729 -1.454  1.016
RIDAGEYR                    -0.039    0.010 -3.871 0.000 -0.059 -0.019
BMXBMI                       0.186    0.026  7.283 0.000  0.136  0.236
group Var                    4.170    0.108                           
======================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The "variance structure parameters" are what distinguish a mixed model
from a marginal model.  Here we only have one such parameter, which is
the variance for groups, estimated to be 4.170.  This means that if we
were to choose two groups at random, their random effects would differ
on average by around 2.89 (this is calculated as the square root of
`2*4.170`).  This is a sizable shift, comparable to the difference
between females and males, or to around 6 years of aging.

Multilevel models can also be used to estimate ICC values.  In the
case of a model with one level, which is what we have here, the ICC is
the variance of the grouping variable (4.170) divided by the sum of
the variance of the grouping variable and the unexplained variance
(154.235).  Note that the unexplained variance is in upper part of the
output, labeled *scale*.  This ratio is around 0.0263, which is very
similar to the estimated ICC obtained using GEE.

__Q4b:__ Include a random intercept for age, and describe your findings.

In [24]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2x)", 
                                re_formula="1+age_cen", groups="group", data=da)
result = model.fit()
result.summary()

/home/helios/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/helios/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/helios/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/helios/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/helios/anaconda3/envs/tensorflow_env/lib/python3.6/sit

<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
======================================================================
Model:                 MixedLM     Dependent Variable:     BPXDI1     
No. Observations:      5100        Method:                 REML       
No. Groups:            30          Scale:                  152.5749   
Min. group size:       105         Likelihood:             -20178.5484
Max. group size:       226         Converged:              No         
Mean group size:       170.0                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                   80.222    4.402 18.224 0.000 71.594 88.850
RIAGENDRx[T.Male]            2.768    0.350  7.917 0.000  2.083  3.453
C(DMDEDUC2x)[T.HS]          -1.036    0.520 -1.993 0.046 -2.054 -0.017
C(DMDEDUC2x)[T.SomeCollege] -0.417    0.483 -0.864 0.387 -1.363  0.529
C(DMDEDUC2x)[T.lt9]         -0.844    0.642 -1.315 0.189 -2.102  0.414
C(DMDEDUC2x)[T.x9_11]       -0.193    0.628 -0.307 0.759 -1.423  1.038
RIDAGEYR                    -0.335    0.088 -3.803 0.000 -0.508 -0.162
BMXBMI                       0.185    0.025  7.258 0.000  0.135  0.235
group Var                    2.073    0.048                           
group x age_cen Cov         -0.005    0.077                           
age_cen Var                  3.129                                    
======================================================================

"""